<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch16-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# 감성 분석
- 데이터셋: IMDb 리뷰 데이터셋(자연어 처리계의 'hello world')
  - IMDb: Internet Movie Database


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
X_train.shape

(25000,)

In [ ]:
X_test.shape

(25000,)

In [ ]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

-----
- 이미 전처리 되어있음
- X_train은 리뷰들의 리스트이고, 각 리뷰는 넘파이 정수 배열로 표현
- 각 정수는 하나의 단어
  - 처리 과정
    - 구두점 제거
    - 모두 소문자
    - 공백으로 나누어 빈도에 따라 인덱싱
  - 정수 0,1,2는 알 수 없는 단어를 의미

In [ ]:
word_index=keras.datasets.imdb.get_word_index()
id_to_word = {id_ +3: word for word, id_ in word_index.items()}

In [ ]:
len(id_to_word) #===단어 개수

88584

In [ ]:
id_to_word[123] #===무작위로 함 추출해봄

'show'

In [ ]:
id_to_word[456]

'lives'

In [ ]:
id_to_word[4]

'the'

- 0,1,2에 단어 추가 해보기

In [ ]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_]=token

In [ ]:
print(id_to_word[0],id_to_word[1], id_to_word[2])

<pad> <sos> <unk>


- X_train 일부 디코딩

In [ ]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

## 텍스트 전처리 방법
- 보통 단어를 인코딩할 때는 구두점, 줄바꿈, 탭 등 많은 글자가 제외됨
- 그리고 공백을 사용해 단어 경계를 구분함
- 하지만 이게 최선은 아님
  - 중국어는 단어 사이에 공백을 사용하지 X
  - 독일어는 여러 단어를 공백 없이 붙여씀
  - 영어도 'San Francisco', 'ILoveDeepLearning'같은 경우가 있음
- 그럼 어떻게?
  - 다쿠 구도가 제안: <strong>부분 단어</strong> 수준으로 텍스트를 토큰화 or 복원하는 비지도 학습 방법
    - 이 방법은 공백을 하나의 문자로 취급하므로 언어 독립적임

In [ ]:
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size=info.splits["train"].num_examples

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


###기본 전처리 함수

In [ ]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch,0,300) #===처음 300글자만 남김 (substring)
  #===정규식을 사용해서 일부 공백으로 변환
  X_batch= tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch= tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  #===리뷰를 공백으로 나눔 -> ragged tensor(중첩 가변 길이 list)반환
  X_batch=tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch #===밀집 텐서로 바꾸고 동일한 길이가 되도록 패딩 토큰 "<pad>"로 모든 리뷰 패딩 

### 어휘 사전 구축

In [ ]:
from collections import Counter
vocabulary =Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3] #=== 가장 많이 등장하는 단어 세 개

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

- 일부만 추출

In [ ]:
vocab_size=10000
truncated_vocabulary=[ word for word, count in vocabulary.most_common()[:vocab_size]]

### 각 단어 인덱싱
  - 13장에서 한 것 처럼 oov(out-of-vocabulary)버킷을 사용하는 룩업 테이블을 만듦

In [ ]:
words=tf.constant(truncated_vocabulary)
word_ids=tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init=tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets=1000
table=tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10770]])>

------
-  'faaaaaaantastic'은 룩업 테이블에 없어서 oov버킷(10000~11000)에 매핑된 것을 확인 

## 최종 훈련 세트 만들기
1. 리뷰를 배치로 묶고
2. preprocess()함수를 사용해 짧은 시퀀스로 바꾸기
3. 룩업 테이블을 사용해 단어 인코딩
4. 배치 프리페치 

In [ ]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch


In [ ]:
train_set= datasets["train"].batch(32).map(preprocess)
train_set=train_set.map(encode_words).prefetch(1)

## 모델 생성 및 훈련


### keras 임베딩 층 적용
[keras_docs](https://keras.io/api/layers/core_layers/embedding/)

In [ ]:
embed_size=128 #==벡터 크기(하이퍼파라미터)
model=keras.models.Sequential([
                               keras.layers.Embedding(vocab_size+num_oov_buckets, embed_size, input_shape=[None]), #=== input: [배치 크기, 타임 스텝 수], output: [배치 크기, 타임 스텝 수, 임베딩 크기]
                               keras.layers.GRU(128, return_sequences=True),
                               keras.layers.GRU(128),#==시퀀스 반환 X 마지막 타임 스텝의 출력만 반환
                               keras.layers.Dense(1,activation='sigmoid')
])


-----------
- 만약 전처리 때 패딩을 했다면, Embedding층을 만들 때 mask_zero=True 매개변수를 추가해야함
  - 그래야 학습할 땐 모델이 패딩 토큰\<pad>을 무시하도록 할 수 있음


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1408000   
_________________________________________________________________
gru (GRU)                    (None, None, 128)         99072     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,606,273
Trainable params: 1,606,273
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 100s 123ms/step - loss: 0.6940 - accuracy: 0.5116
Epoch 2/5
782/782 [==============================] - 95s 121ms/step - loss: 0.5232 - accuracy: 0.7413
Epoch 3/5
782/782 [==============================] - 95s 122ms/step - loss: 0.3304 - accuracy: 0.8593
Epoch 4/5
782/782 [==============================] - 95s 122ms/step - loss: 0.2357 - accuracy: 0.9085
Epoch 5/5
782/782 [==============================] - 96s 123ms/step - loss: 0.1656 - accuracy: 0.9399


### 사전훈련된 임베딩 재사용하기
- 위처럼 단어 임베딩을 직접 훈련할 수도 있지만, 사전 훈련된 임베딩을 재사용할 수도 있음
- [TF 허브 저장소](https://tfhub.dev/)에서 원하는 모듈을 찾아서 활용하면됨
  - tensorflow_hub.KerasLayer 층이 모듈과 사전 훈련된 가중치를 자동으로 다운해서 모델에 포함시켜줌
    - 해당 층은 훈련되지 않음(근데 trainable=True하면 미세 조정할 순 있음)

In [ ]:
import tensorflow_hub as hub
model_hub= keras.Sequential([
                             hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", dtype=tf.string, input_shape=[], output_shape=[50]),
                             keras.layers.Dense(128, activation="relu"),
                             keras.layers.Dense(1,activation="sigmoid")
                             
])

In [ ]:
model_hub.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 48,197,257
Trainable params: 6,657
Non-trainable params: 48,190,600
_________________________________________________________________


# 신경망 기계 번역(NMT)을 위한 인코더-디코더 네트워크
<img src="https://user-images.githubusercontent.com/13735079/34563713-d9b26bb6-f13a-11e7-8c12-a1830a397d78.png"/>

- 책에 있는 그림(위 그림)을 보면 인코더 input 주입 시에 <strong>문장을 뒤집어서</strong> 넣는 것을 알 수 있는데,
 - 책에선 '문장의 시작 부분을 인코더에 마지막으로 주입한다. 디코더가 번역할 첫 부분이기 때문이다'라고 하고 있음
- 근데 인터넷을 찾아보면 뒤집어서 주입하는 경우를 찾아볼 수 없었음
- 그 이유는, <strong>인코더에 어떤 RNN</strong>을 쓰느냐에 따라 다르기 때문
  - <strong>Bi-Directional RNN</strong>
    - 왼쪽 -> 오른쪽 
    - 오른쪽 -> 왼쪽<br><br>
    <img src="https://miro.medium.com/max/1088/1*sBZq013xGoFvyH5e0sXtdg.png"/>
<br><br>
  - <strong>Uni-directional RNN</strong>
    - 왼쪽 -> 오른쪽
- 즉, 책에 있는 그림은 Uni-directional RNN이라서 뒤집어서 넣는게 의미가 있었던 것이고,
- 주로 NMT에서 많이 쓰는건 Bi-Directional RNN이라서 인터넷에 굳이 input을 뒤집은게 안보인거였음

참조)
- [github_issues](https://github.com/tensorflow/nmt/issues/231)

## 기본적인 인코더 - 디코더 모델

### 텐서플로 애드온(Addon)
- 텐서플로우에서 제공하지 않는 기능을 제공하는 API
  - 시퀀스 투 시퀀스 관련 모듈도 있음
  - [seq2seq module](https://www.tensorflow.org/addons/api_docs/python/tfa/seq2seq)


In [ ]:
pip install tensorflow_addons

     |████████████████████████████████| 706kB 7.2MB/s 


In [ ]:
import tensorflow_addons as tfa

In [ ]:
#===인풋 정의
vocab_size, embed_size=10000, 128
encoder_inputs=keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs=keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths=keras.layers.Input(shape=[], dtype=np.int32)

#===임베딩 정의
embeddings=keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings= embeddings(encoder_inputs)
decoder_embeddings=embeddings(decoder_inputs)

#===인코더 정의
encoder=keras.layers.LSTM(512, return_state=True) #최종 은닉 상태를 디코더로 보내기 위해 return_state=True
encoder_output, state_h, state_c = encoder(encoder_embeddings)
encoder_state=[state_h, state_c] #==단기 & 장기 은닉 상태

#===샘플러의 역할: 각 스텝에서 디코더에게 이전 스텝의 출력이 무엇인지 알려줌
sampler=tfa.seq2seq.sampler.TrainingSampler()

#===디코더 정의
decoder_cell=keras.layers.LSTMCell(512)
output_layer=keras.layers.Dense(vocab_size)
decoder=tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, 
    initial_state=encoder_state,
    sequence_length=sequence_lengths)

Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model2 = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [ ]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 128)    1280000     input_37[0][0]                   
                                                                 input_38[0][0]                   
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, 512), (None, 1312768     embedding_12[0][0]         

## 양방향(Bi-Directional) RNN
-  위에서 살짝 다뤘음
- <strong>일반적인 순환 층은 과거와 현재의 입력</strong>만 보고 출력을 생성
  - 즉, <strong>미래</strong>를 볼 수 없음
- 시계열 예측에는 괜찮지만, NMT같은 여러 NLP작업에는 적합하지 않음
  - NLP작업은 주어진 단어를 인코딩하기 전에 <strong>다음 단어를 미리 봐야 문맥 이해 가능</strong>
- 양방향 RNN은<strong> 과거 -> 미래</strong> 방향으로도 보고, <strong>미래 -> 과거</strong> 방향으로도 보는 것이다.

<img src="https://i.imgur.com/XGpeu82.png" width=60% height=60% />




### 구현

In [ ]:
#== 양방향 순환 층 구현 방법 -> Bidirectional 층으로 순환 층을 감싸면 됨
keras.layers.Bidirectional(keras.layers.GRU(10
                                            , return_sequences=True))

---------
- Bidirectional층은 순환 층(여기선 GRU층)을 반대 방향으로 복사함
- 그 다음 두 층에서 나온 출력을(여기선 타임 스텝마다 10 * 2개) 연결 

## 빔 검색
- 디코더는 스텝마다 무조건 <strong>가장 가능성 있는 단어를 출력(greedy하게)</strong>함
- 하지만 <strong>항상 최적</strong>의 번역을 만들진 못함
- 모델의 <strong>앞선 실수</strong>를 고치게 하는 방법 중 하나:<strong> Beam Search</strong>
  1. <strong>k개의 가능성 있는</strong>( ex) 상위 세 개) 문장의 리스트를 유지

  2. 디코더 스텝마다 각 문장의 단어를 하나씩 생성하여 가능성 있는 k개의 문장을 만듦<br> 
    *k: 빔 너비(beam width) 

<br>

ex) 프랑스어 'Comment vas-tu?'를 영어('How are you going?')로 번역하기, k=3   
- 1 step: 최상위 k개(3개)의 문장을 담은 리스트 생성['How'(75% 추정 확률), 'What'(3%), 'You(1%)']
- 2 step: 3개의 모델로 복사해서 각 문장의 다음 단어 찾기
  - 모델1: 'How'문장 뒤에 이어질 단어 찾기 -> 'will(36%)', 'are(32%)', 'do(16%)'... <br> *어휘 사전에 있는 단어에 해당하는 추정확률을 모두 구함<br>
  - 모델2: 'What' 문장 뒤에 이어질 단어 찾기<br>
  - 모델3: 'You' 문장 뒤에 이어질 단어 찾기<br>
- 3 step: 각 모델이 현재까지의 단어(두개의 단어)로 이루어진 문장에 대해 확률 계산 + 상위 k개(3개)만 추리기
  - ex) 'How will' -> 0.75 * 0.36 = 0.27<br>
    이런 식으로 구하면 'How will(27%)' >  'How are(24%)' > 'How do(12%)'로, <strong>'How are'이 배제되지 않음</strong><br> *기존의 방식대로 했으면 'How' 선택 후 'will'을 선택하고 넘어감
- 위 과정 반복


<br><br>
참조)
- [velog- Beam search in NLP](https://velog.io/@nawnoes/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-Beam-Search) 

### 구현

In [ ]:
beam_width=10
decoder=tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_width, output_layer=output_layer
)

decoder_initial_state= tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width
)


# 어텐션 메커니즘
-  인코더-디코더 네트워크의 문제점
  - 빔 서치 등의 방법으로 성능을 개선하더라도 긴 문장에서는 성능이 나쁨
    - 특히 인코딩 후 나오는 압축된 정보 때문

- 따라서 <strong>중요한 단어에 집중</strong>해서 디코더에 전달하는 기법인 <strong>'어텐션'</strong> 등장 
- 어텐션은 기존 방식보다 더 많은 데이터를 디코더에 전달함
  - 입력 인코더의 마지막 은닉 상태만 디코더에 보내는 것이 아닌, 
  - 인코더의 <strong>모든 출력을 디코더로 전송함</strong>

  <img src="https://miro.medium.com/max/3820/1*wnXVyE8LXPfODvB_Z5vu8A.jpeg" width=80% height=80%/>

  - 각 타임 스텝에서 디코더의 메모리 셀은 모든 인코더 출력의 가중치 합을 계산함
    - 이 단계에서 주의를 집중할 단어를 결정함
    - softmax 이후 나온 값(a)이 클 수록 더 많은 주의를 기울일 단어
  - 위 어텐션 층은 인코더-디코더 모델의 나머지 부분과 함께 훈련됨
  
<br>

- 장점
  - 응용력
    - 비주얼 어텐션
      - NMT 외에도 이미지 캡션 생성 문제에도 적용 가능
    - 트랜스포머 구조
      - 순환층이나 합성곱을 사용하지 않고 어텐션 메커니즘만 사용한 모델(NMT문제에서 성능 크게 향상)
    - etc. 

  - 설명 가능성
    - 어텐션의 장점은 <strong>모델이 어떤 출력을 만들도록 이끄는 것이 무엇인지 이해하기 쉽다</strong>는 것
    - 이는 모델이 올바르게 예측하지 못할 때 특히 유용<br>
    ex) 눈 위를 걷는 강아지 사진에 '눈 위를 걷는 늑대'라는 캡션 생성 시, <br>'늑대'라는 단어를 출력할 때 초점을 맞춘 것이 무엇인지 확인할 수 있다.<br>(강아지 뿐만 아니라 눈에도 주의를 기울이고 있을지 모름)



<br>

  참조)
  - 핸즈온 머신러닝
  - 밑바닥부터 시작하는 딥러닝2
  - [glee1228님 tistory](https://glee1228.tistory.com/3)
  - [medium](https://medium.datadriveninvestor.com/attention-in-rnns-321fbcd64f05)

## 구현
-  인코더-디코더 모델에 루옹 어텐션 추가하기
  - 바흐다나우 어텐션(연결 어텐션), 루옹 어텐션 등 여러가지

In [ ]:
def make_attention(units, n_units, encoder_sequence_length):
  attention_mechanism = tfa.seq2seq.attention_wrapper.LuongAttention(
      units, encoder_state, memory_sequence_length=encoder_sequence_length
  )

  attention_decoder_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(
      decoder_cell, attention_mechanism, attention_layer_size=n_units
  )
  return [attention_mechanism, attention_decoder_cell]

# 트랜스포머 구조: 어텐션이 필요한 전부다
- 어텐션 메커니즘만 사용해서 NMT문제에서 최고 수준 성능을 크게 향상시킴
- 빠른 훈련 , 쉬운 병렬화

<img src="https://blog.kakaocdn.net/dn/AbY3k/btqS1VZlsu0/a7YCVUTxAxF2mANkVZWoo0/img.png" />

- 모든 층은 타임 스텝에 독립적이다. 따라서 <strong>각 단어는 모든 다른 단어에 대해 독립적으로 처리</strong>된다.
- <strong>하지만 어떻게 한 번에 한 단어씩 보면서 문장을 번역할까?</strong>
  - 주요 구성 요소
    - <strong>인코더의 멀티-헤드 어텐션</strong>
      - 관련이 많은 단어에 더 많은 주의를 기울이며 <strong>각 단어와 동일한 문장에 있는 다른 단어의 관계</strong>를 인코딩함
      - ex) 'They welcomed the Queen of the United Kingdom'
        - 단어 'Queen'에 대해 'They'나 'welcomed'보단 'United'와 'Kingdom'에 더 주의를 기울인다.

    - <strong>마스크드 멀티-헤드 어텐션</strong>
      - 위 멀티-헤드 어텐션과 동일한 작업을 수행하지만, <strong>이전에 등장한 단어에만</strong> 주의를 기울인다.

    - <strong>디코더의 위쪽 멀티-헤드 어텐션</strong> 
      - 디코더가 <strong>입력 문장에 있는 단어</strong>에 주의를 기울인다.
      - ex) 디코더가 'Queen'을 번역할 때, 입력 문장에 있는 단어'Queen'에 가장 주의를 기울인다. 

    - <strong>위치 인코딩</strong>
      - 문장에 있는 <strong>단어의 위치</strong>를 나타내는 단순한 밀집 벡터
        - 멀티-헤드 어텐션 층은 단어 사이의 관계만 보고, 단어의 순서나 위치를 고려하지 않으므로 필요
      - n번째 위치 인코딩이 각 문장에 있는 n번째 단어의 단어 임베딩에 더해짐
        - 이를 통해 모델이 각 단어의 위치를 알 수 있음
      

<br><br>
# 연습문제

## 상태가 없는 RNN 대비 상태가 있는 RNN의 장단점은 무엇인가요?
- 장점
  - <strong>장기 패턴</strong>을 감지할 수 있다.
    - 상태가 없는 RNN은 훈련한 윈도우 크기와 같거나 작은 길이의 패턴만 감지 가능
- 단점
  - <strong>구현</strong>이 어렵다.
  - <strong>데이터셋</strong>을 적절히 준비하는 것이 어렵다.
  


## 왜 자동 변역에 시퀀스-투-시퀀스 RNN 대신 인코더-디코더 RNN을 사용하나요?
- 일반적으로 <strong>문장을 읽으면서</strong> 단어 하나씩 번역하면 결과가 매우 좋지 않음 (시퀀스-투-시퀀스)
- <strong>전체 문장을 읽고 난 후</strong> 번역하는 것이 훨씬 결과가 좋음 (인코더-디코더 RNN)

## 가변 길이 입력 시퀀스를 어떻게 다룰 수 있나요? 가변 길이 출력 시퀀스는 어떤가요?
- <strong>패딩</strong>
  1. 배치에 있는 모든 시퀀스 길이가 동일하도록 짧은 시퀀스에 패딩을 추가한다.
  2. RNN이 패딩 토큰을 무시하도록 마스킹 처리를 하도록 한다.

- 더 좋은건, 길이가 <strong>비슷한 시퀀스를 모아</strong> 배치를 만드는 것

## 빔 검색이 무엇인가요? 왜 사용해야 하나요? 이를 구현하기 위해 어떤 도구를 사용할 수 있나요?
- 훈련된 인코더-디코더 모델의 성능을 향상하기 위해 사용하는 기술
  - 하나의 문장을 늘리기 위해 각 스텝에서 가장 가능성 있는 다음 단어를 탐욕적으로 선택하기 보다,
  -  <strong>가능성 있는 문장 k개를 동시에 탐색</strong> 가능
    - 파라미터 k를 '빔 너비'라 부름
    - k가 클수록 더 많은 연산 요구, 더 많은 메모리 차지
    - 하지만 k가 클수록 더 정확한 시스템이 될 수 있을 것
  - 해당 기법은 병렬화하기 쉬움
- 텐서플로 애드온을 사용하면 빔 검색 구현이 쉬움

## 어텐션 메커니즘이 무엇인가요? 어떤 장점이 있나요?
- 디코더가 입력 시퀀스에 직접 접근하기 위해 사용된 기법
- <strong>디코더 타임 스텝마다 현재 디코더의 상태와 인코더의 전체 출력을</strong> 정렬 모델(alignment model)이 처리하여 <strong>입력 타임스텝에 대한 정렬 점수</strong>를 출력
  - 이 점수는 <strong>입력의 어느 부분이 현재 디코더의 타임 스텝에 가장 관련</strong>되어 있는지 나타냄
  - 인코더 출력(정렬 점수로 가중치된)의 가중치 합이 디코더로 주입되어 다음 디코더 상태와 해당 스텝의 출력을 만듦

- 장점
  - <strong>긴 입력 시퀀스</strong>에 대한 성능 향상
  - 정렬 점수 덕분에 모델을 <strong>디버깅</strong>하고 <strong>해석</strong>하기 용이
  - 멀티-헤드 어텐션 층을 사용하는 <strong>트랜스포머 구조</strong>의 핵심


## 트랜스포머 구조에서 가장 중요한 층이 무엇인가요? 이 층의 목적이 무엇인가요?
- 가장 중요한 층: <strong>멀티-헤드 어텐션 층</strong>
  - 이 층의 목적: 모델이 <strong>어떤 단어가 서로 관련</strong>되어 있는지 구별하도록 돕는 것 -> 단어 표현 향상